
# Bandido Multibrazo (epsilon–greedy)

Este notebook es **auto-contenido** y reproduce el caso práctico de optimización de un banner con 5 imágenes (5 brazos)

Responde automáticamente a las 3 preguntas del enunciado:

1. **¿Cuál es el valor de ε que maximiza la imagen con mayor número de clics?**  
2. **¿Cuál es la imagen que más clics obtiene y cuántos?**  
3. **¿Qué ocurre con explotación 100% (ε = 0)?**

In [2]:
import numpy as np
import pandas as pd

# Para reproducibilidad
rng = np.random.default_rng(42)

# Probabilidades "reales" de click por imagen (brazos)
TRUE_PROBS = np.array([0.1, 0.6, 0.2, 0.1, 0.3])  # 5 imágenes
N_ARMS = len(TRUE_PROBS)

def bernoulli(p, rng):
    return 1 if rng.random() < p else 0

In [3]:
def multi_armed_bandit(num_games=1000, epsilon=0.1, rng=None, true_probs=None):
    """
    Epsilon-greedy clásico para bandido multibrazo.
    Devuelve:
      - bandits: probabilidades reales (array)
      - total_reward: clics totales (int)
      - q_bandits: estimación Q(a) final por brazo (array float)
      - num_selected_bandit: cuántas veces se eligió cada brazo (array int)
      - clicks_por_brazo: clics logrados por brazo (array int)
    """
    if rng is None:
        rng = np.random.default_rng()
    if true_probs is None:
        true_probs = TRUE_PROBS

    n_arms = len(true_probs)
    q = np.zeros(n_arms, dtype=float)              # estima recompensa esperada por brazo
    n = np.zeros(n_arms, dtype=int)                # conteo de selecciones por brazo
    clicks = np.zeros(n_arms, dtype=int)           # clics acumulados por brazo
    total_reward = 0

    for t in range(num_games):
        # selección epsilon-greedy
        if rng.random() < epsilon:
            a = rng.integers(0, n_arms)           # explorar
        else:
            a = int(np.argmax(q))                 # explotar

        # generar recompensa (click) ~ Bernoulli(true_probs[a])
        r = 1 if rng.random() < true_probs[a] else 0

        # actualizar contadores y Q(a)
        n[a] += 1
        clicks[a] += r
        total_reward += r
        # actualización incremental de la media
        q[a] += (r - q[a]) / n[a]

    return (np.array(true_probs), int(total_reward), q.copy(), n.copy(), clicks.copy())


## Ejecución de ejemplo (una corrida)

In [4]:

bandits, total_reward, q_bandits, num_selected_bandit, clicks_por_brazo = multi_armed_bandit(
    num_games=1000, epsilon=0.1, rng=np.random.default_rng(123)
)
print("Probabilidades reales:", bandits)
print("Clicks totales:", total_reward)
print("Q finales:", np.round(q_bandits, 3))
print("Veces elegido:", num_selected_bandit)
print("Clicks por imagen:", clicks_por_brazo)


Probabilidades reales: [0.1 0.6 0.2 0.1 0.3]
Clicks totales: 544
Q finales: [0.068 0.6   0.125 0.    0.238]
Veces elegido: [ 44 889  24  22  21]
Clicks por imagen: [  3 533   3   0   5]


## P1) Barrido de ε para maximizar clics del mejor brazo

In [5]:

eps_grid = [0.0, 0.01, 0.05, 0.1, 0.15, 0.2, 0.3, 0.5]
rep = 50          # repeticiones por epsilon para estabilidad
num_games = 1000

rows = []
for eps in eps_grid:
    for rseed in range(rep):
        out = multi_armed_bandit(num_games=num_games, epsilon=eps, rng=np.random.default_rng(rseed))
        bandits, total_reward, qB, n_sel, clicks = out
        rows.append({
            "epsilon": eps,
            "mejor_imagen_idx": int(np.argmax(clicks)),
            "clicks_mejor_imagen": int(np.max(clicks)),
            "total_clicks": int(total_reward)
        })

df = pd.DataFrame(rows)

resumen = (df.groupby("epsilon", as_index=False)
             .agg(clicks_mejor_brazo_prom=("clicks_mejor_imagen", "mean"),
                  total_clicks_prom=("total_clicks", "mean"),
                  best_arm_mode=("mejor_imagen_idx", lambda x: x.mode().iat[0])))

# Epsilon que maximiza la métrica principal (promedio de clicks del mejor brazo)
fila_top = resumen.sort_values("clicks_mejor_brazo_prom", ascending=False).iloc[0]
epsilon_opt = float(fila_top["epsilon"])
clicks_mejor_brazo_prom = float(fila_top["clicks_mejor_brazo_prom"])
total_clicks_prom = float(fila_top["total_clicks_prom"])
best_arm_mode = int(fila_top["best_arm_mode"])

print("Tabla resumen por ε:")
display(resumen)

print("\nRespuesta P1 → ε óptimo:", epsilon_opt)
print("   Promedio clicks del mejor brazo:", round(clicks_mejor_brazo_prom, 2))
print("   Promedio clicks totales:", round(total_clicks_prom, 2))
print("   Brazo más frecuente como 'mejor' (modo):", best_arm_mode, "(indexado desde 0; imagen humana =", best_arm_mode+1, ")")


Tabla resumen por ε:


,epsilon,clicks_mejor_brazo_prom,total_clicks_prom,best_arm_mode
0,0.00,100.80,100.80,0
1,0.01,253.28,294.68,1
2,0.05,463.26,498.10,1
3,0.10,491.78,521.60,1
4,0.15,486.98,517.84,1
5,0.20,479.94,513.28,1
6,0.30,441.52,487.92,1
7,0.50,352.48,426.22,1



Respuesta P1 → ε óptimo: 0.1
   Promedio clicks del mejor brazo: 491.78
   Promedio clicks totales: 521.6
   Brazo más frecuente como 'mejor' (modo): 1 (indexado desde 0; imagen humana = 2 )


## P2) Imagen que más clics obtiene y cuántos (con ε óptimo)

In [6]:

# Ejecutamos varias veces con epsilon_opt y tomamos la mejor corrida observada
mejor_corrida = None
best_clicks = -1

for rseed in range(200):
    out = multi_armed_bandit(num_games=num_games, epsilon=epsilon_opt, rng=np.random.default_rng(10_000 + rseed))
    bandits, total_reward, qB, n_sel, clicks = out
    if int(np.max(clicks)) > best_clicks:
        best_clicks = int(np.max(clicks))
        mejor_corrida = out

bandits, total_reward, qB, n_sel, clicks = mejor_corrida
imagen_top_idx = int(np.argmax(clicks))
imagen_top_hum = imagen_top_idx + 1  # 1..5 para reportar como "imagen #"

print("Respuesta P2 → Imagen que más clics obtiene (con ε óptimo):")
print("   Imagen (index Python):", imagen_top_idx,  "→ Imagen humana:", imagen_top_hum)
print("   Clics de esa imagen:", int(np.max(clicks)))
print("   Clicks por imagen:", clicks.tolist())
print("   Veces elegida por imagen:", n_sel.tolist())
print("   Q finales:", np.round(qB, 3).tolist())


Respuesta P2 → Imagen que más clics obtiene (con ε óptimo):
   Imagen (index Python): 1 → Imagen humana: 2
   Clics de esa imagen: 577
   Clicks por imagen: [6, 577, 2, 4, 5]
   Veces elegida por imagen: [31, 915, 17, 19, 18]
   Q finales: [0.194, 0.631, 0.118, 0.211, 0.278]


## P3) Solo explotación (ε = 0)

In [7]:

out_explot = multi_armed_bandit(num_games=num_games, epsilon=0.0, rng=np.random.default_rng(999))
bandits_e0, total_reward_e0, qB_e0, n_sel_e0, clicks_e0 = out_explot

print("Respuesta P3 → Resultados con ε = 0 (100% explotación)")
print("   Mejor imagen (argmax clicks):", int(np.argmax(clicks_e0)), "→ Imagen humana:", int(np.argmax(clicks_e0))+1)
print("   Clicks de esa imagen:", int(np.max(clicks_e0)))
print("   Clicks totales:", int(total_reward_e0))
print("   Clicks por imagen:", clicks_e0.tolist())
print("   Veces elegida por imagen:", n_sel_e0.tolist())
print("   Q finales:", np.round(qB_e0, 3).tolist())

print("\nInterpretación: con ε=0 no hay exploración; el algoritmo puede atascarse en una elección temprana "
      "y no identificar el verdadero mejor brazo si las primeras muestras no fueron representativas.")


Respuesta P3 → Resultados con ε = 0 (100% explotación)
   Mejor imagen (argmax clicks): 0 → Imagen humana: 1
   Clicks de esa imagen: 90
   Clicks totales: 90
   Clicks por imagen: [90, 0, 0, 0, 0]
   Veces elegida por imagen: [1000, 0, 0, 0, 0]
   Q finales: [0.09, 0.0, 0.0, 0.0, 0.0]

Interpretación: con ε=0 no hay exploración; el algoritmo puede atascarse en una elección temprana y no identificar el verdadero mejor brazo si las primeras muestras no fueron representativas.


## Resumen listo

In [8]:

print("P1) ε óptimo:", epsilon_opt, 
      "| Promedio clicks mejor brazo:", round(clicks_mejor_brazo_prom, 2),
      "| Promedio clicks totales:", round(total_clicks_prom, 2),
      "| Brazo más frecuente como mejor:", best_arm_mode, "(humano =", best_arm_mode+1, ")")

print("P2) Con ε óptimo → Imagen (humana) con más clics y cantidad arriba (ver bloque P2).")
print("P3) Con ε = 0 → ver bloque P3 (solo explotación).")


P1) ε óptimo: 0.1 | Promedio clicks mejor brazo: 491.78 | Promedio clicks totales: 521.6 | Brazo más frecuente como mejor: 1 (humano = 2 )
P2) Con ε óptimo → Imagen (humana) con más clics y cantidad arriba (ver bloque P2).
P3) Con ε = 0 → ver bloque P3 (solo explotación).
